In [1]:
# Boltzmann Machines

# Importing the libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

#Dataset from https://grouplens.org/datasets/movielens/

# Importing the dataset
!curl https://raw.githubusercontent.com/muke888/UdemyDeepLearning/master/5.%20Boltzmann%20Machines/Datasets/movies.dat -o movies.dat
!curl https://raw.githubusercontent.com/muke888/UdemyDeepLearning/master/5.%20Boltzmann%20Machines/Datasets/users.dat -o users.dat
!curl https://raw.githubusercontent.com/muke888/UdemyDeepLearning/master/5.%20Boltzmann%20Machines/Datasets/ratings.dat -o ratings.dat
!curl https://raw.githubusercontent.com/muke888/UdemyDeepLearning/master/5.%20Boltzmann%20Machines/Datasets/u1.base -o u1.base
!curl https://raw.githubusercontent.com/muke888/UdemyDeepLearning/master/5.%20Boltzmann%20Machines/Datasets/u1.test -o u1.test

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  167k  100  167k    0     0  1026k      0 --:--:-- --:--:-- --:--:-- 1026k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  131k  100  131k    0     0   898k      0 --:--:-- --:--:-- --:--:--  898k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 23.4M  100 23.4M    0     0  43.0M      0 --:--:-- --:--:-- --:--:-- 43.0M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1549k  100 1549k    0     0  5913k      0 --:--:-- --:--:-- --:--:-- 5891k
  % Total    % Received % Xferd  Average Speed   Tim

In [0]:
# Importing the Dataset1
# These files contain 1,000,209 anonymous ratings of approximately 3,900 movies 
# made by 6,040 MovieLens users who joined MovieLens in 2000.

movies = pd.read_csv('movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

In [3]:
# MovieID::Title::Genres
movies.head()

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
# UserID::Gender::Age::Occupation::Zip-code
users.head()

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [5]:
# UserID::MovieID::Rating::Timestamp
ratings.head()

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [0]:
# Dataset2
# Using 100,000 ratings (1-5) from 943 users on 1682 movies.
# Each user has rated at least 20 movies. 
# Simple demographic info for the users (age, gender, occupation, zip)
# 80%/20% split

# Preparing the training set and the test set
training_setDF = pd.read_csv('u1.base', delimiter = '\t')
training_set = np.array(training_setDF, dtype = 'int')
test_setDF = pd.read_csv('u1.test', delimiter = '\t')
test_set = np.array(test_setDF, dtype = 'int')

In [7]:
# user id | item id | rating | timestamp.
training_setDF.info()
training_setDF.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79999 entries, 0 to 79998
Data columns (total 4 columns):
1            79999 non-null int64
1.1          79999 non-null int64
5            79999 non-null int64
874965758    79999 non-null int64
dtypes: int64(4)
memory usage: 2.4 MB


,1,1.1,5,874965758
0,1,2,3,876893171
1,1,3,4,878542960
2,1,4,3,876893119
3,1,5,3,889751712
4,1,7,4,875071561


In [8]:
test_setDF.info()
test_setDF.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19999 entries, 0 to 19998
Data columns (total 4 columns):
1            19999 non-null int64
6            19999 non-null int64
5            19999 non-null int64
887431973    19999 non-null int64
dtypes: int64(4)
memory usage: 625.0 KB


,1,6,5,887431973
0,1,10,3,875693118
1,1,12,5,878542960
2,1,14,5,874965706
3,1,17,3,875073198
4,1,20,4,887431883


In [9]:
# Getting the number of users and movies
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))
print (nb_users, nb_movies)

943 1682


In [0]:
# Converting the data into an array with users in lines and movies in columns

def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[:,1][data[:,0] == id_users]
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data
  

training_set = convert(training_set)
test_set = convert(test_set)

In [0]:
# Converting the data into Torch tensors
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [0]:
# Converting the ratings into binary ratings 1 (Liked) or 0 (Not Liked)
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1
test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

In [0]:
# Creating the architecture of the Neural Network

class RBM():
    def __init__(self, nv, nh):
        self.W = torch.randn(nh, nv)
        self.a = torch.randn(1, nh)
        self.b = torch.randn(1, nv)
    def sample_h(self, x):
        wx = torch.mm(x, self.W.t())
        activation = wx + self.a.expand_as(wx)
        p_h_given_v = torch.sigmoid(activation)
        return p_h_given_v, torch.bernoulli(p_h_given_v)
    def sample_v(self, y):
        wy = torch.mm(y, self.W)
        activation = wy + self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)
    def train(self, v0, vk, ph0, phk):
        self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
        self.b += torch.sum((v0 - vk), 0)
        self.a += torch.sum((ph0 - phk), 0)
        
nv = len(training_set[0])
nh = 100
batch_size = 100
rbm = RBM(nv, nh)

In [16]:
# Training the RBM via RMSE method

nb_epoch = 10
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(0, nb_users - batch_size, batch_size):
        vk = training_set[id_user:id_user+batch_size]
        v0 = training_set[id_user:id_user+batch_size]
        ph0,_ = rbm.sample_h(v0)
        for k in range(10):
            _,hk = rbm.sample_h(vk)
            _,vk = rbm.sample_v(hk)
            vk[v0<0] = v0[v0<0]
        phk,_ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += np.sqrt(torch.mean((v0[v0>=0] - vk[v0>=0])**2)) # RMSE here
        s += 1.
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

epoch: 1 loss: tensor(0.5938)
epoch: 2 loss: tensor(0.4984)
epoch: 3 loss: tensor(0.5000)
epoch: 4 loss: tensor(0.4970)
epoch: 5 loss: tensor(0.4989)
epoch: 6 loss: tensor(0.5019)
epoch: 7 loss: tensor(0.4967)
epoch: 8 loss: tensor(0.4978)
epoch: 9 loss: tensor(0.5000)
epoch: 10 loss: tensor(0.4972)


In [17]:
# Testing the RBM via RMSE method

test_loss = 0
s = 0.
for id_user in range(nb_users):
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += np.sqrt(torch.mean((vt[vt>=0] - v[vt>=0])**2)) # RMSE here
        s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.4628)


In [18]:
# Training the RBM via Average Distance method

b_epoch = 10
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(0, nb_users - batch_size, batch_size):
        vk = training_set[id_user:id_user+batch_size]
        v0 = training_set[id_user:id_user+batch_size]
        ph0,_ = rbm.sample_h(v0)
        for k in range(10):
            _,hk = rbm.sample_h(vk)
            _,vk = rbm.sample_v(hk)
            vk[v0<0] = v0[v0<0]
        phk,_ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0])) # Average Distance here
        s += 1.
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

epoch: 1 loss: tensor(0.2471)
epoch: 2 loss: tensor(0.2515)
epoch: 3 loss: tensor(0.2457)
epoch: 4 loss: tensor(0.2477)
epoch: 5 loss: tensor(0.2473)
epoch: 6 loss: tensor(0.2514)
epoch: 7 loss: tensor(0.2457)
epoch: 8 loss: tensor(0.2469)
epoch: 9 loss: tensor(0.2486)
epoch: 10 loss: tensor(0.2491)


In [19]:
# Testing the RBM via Average Distance method

test_loss = 0
s = 0.
for id_user in range(nb_users):
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0])) # Average Distance here
        s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.2595)


In [20]:
#we obtained an Average Distance of 0.25,
# which is equivalent to about 75% of correct prediction.

#If you want to check that 0.25 corresponds to 75% of success, you can run the following test:

import numpy as np
u = np.random.choice([0,1], 100000)
v = np.random.choice([0,1], 100000)
u[:50000] = v[:50000]
sum(u==v)/float(len(u)) # -> you get 0.75
np.mean(np.abs(u-v)) # -> you get 0.25

0.25152